## Data Validator
Notebook that reads in old scraped job advertisement data and checks for missing information.

If there is missing informtion, this notebook has the functionality to fill it in.

In [ ]:
import glob
import os
import requests

from bs4 import BeautifulSoup
from lxml import etree

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# This information got a bit mangled in Cameron's original code, so we'll create a extra 
# variable to hold these strings in case they change in the future.
date_posted_str= "Date Posted"
job_url_str = "Job URL"

data_file_directory = "../data"

In [ ]:
# TODO better to move all this stuff into its own module and import it.

In [ ]:
def get_csv_files(data_directory):
    r"""Gets the path to all data files in directory
    
    Gets the relative file path to all the data files in a specificed directory.
    """
    csv_file_path = os.path.join(data_directory, "*.csv")
    data_file_paths = glob.glob(csv_file_path)
    return data_file_paths

In [ ]:
def get_urls_missing_date_posted(df):
    r"""Gets urls for jobs with missing date posted data.

    Reads in DataFrame and looks for missing date posted information, then returns a 
    DataFrame with all the urls to the jobs with missing date posted information.

    Parameters
    ----------
    df : DataFrame
        DataFrame containing a column of data with date posted information and a column
        of information with original request url information.
    
    Returns
    -------
    Series
        Pandas Series (with ID) of urls to pages where posted date was missed.
    """
    no_dates = df[df[date_posted_str].isna()]
    urls = no_dates.loc[:,job_url_str]
    return urls


In [ ]:
def has_missing_dates(df):
    """Checks if DataFrame has missing date posted information.
    
    Paramters
    ---------
    df : DataFrame
        DataFrame containing a column of data with date posted information and a column
        of information with original request url information.
    
    Returns
    -------
    bool
        True if there are missing date posted values. False if not.
    """
    return  df[date_posted_str].isnull().sum() > 0

In [ ]:
def get_page_from_url(url):
    r"""Makes request for page.


    Parameters
    ----------
    url : str
        Url of page to request.

    Returns
    -------
    requests.models.Response
        A request Response object.

        Access the page content using .content.
    """
    return requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

def get_page_content(url):
    r"""Gets page content from a url
    
    Parameters
    ----------
    url : str
        Url of page to get content of.

    Returns
    -------
    bytes
        Page content of Response object in bytes. 

        Parsing the bytes of the page content is suppored by BeautifulSoup.
    """
    page = get_page_from_url(url)
    return page.content
    

In [ ]:
def get_date_posted(page_content):
    r"""Parses date posted information from page content.
    
    
    Parameters
    ----------
    page_content : str
        String of page HTML dom.

    Returns
    -------
    str
        The date posted information from the page content.
    """
    soup = BeautifulSoup(page_content, "html.parser")
    dom = etree.HTML(str(soup))
    date_xpath = '//*[@id="main"]/div/div[3]/div[1]/div[1]/div[1]/div/div[1]/dl/dd[4]'
    date_element = dom.xpath(date_xpath)
    return date_element[0].text    

In [ ]:
def standardize_date_string(date_posted_str):
    r"""Takes date posted string from page content and converts it to desired format.
    
    Converts date string to the form yyyy-mm-dd. Date string format may change overtime.
    
    Parameters
    ----------
    date_posted_str : str
        String of the date posted from the website content.

    Returns
    -------
    Timestamp
        Standardized date posted information.
    """
    return pd.to_datetime(date_posted_str, infer_datetime_format=True, format="%Y/%m/%d")

In [ ]:
def get_date_posted_from_url(url):
    r"""Get standardized date posted time from a job url
    
    Parameters
    ----------
    url : str
        Url of page to get content of.

    Returns
    -------
    Timestamp
        Standardized date posted information for a job listing.
    """
    page_content = get_page_content(url)
    date = get_date_posted(page_content)
    return standardize_date_string(date)

In [ ]:
def fill_in_missing_posting_dates_form_csv(csv_data, output_file_name, output_path):
    r"""Fills in missing date information for csv file.

    Fills in missing date posted information from `csv_data`, and saves
    updated file of same name as input to specified directory.

    Parameters
    ----------
    csv_data : DataFrame
        DataFrame containing a column of data with date posted information and a column
        of information with original request url information.

    output_file_name : str
        Filename to save file under

    output_path : str
        Output directory path to write the csv files to.
    
    """
    output_path = os.path.join(output_path, output_file_name)

    missing_posted_date_urls = get_urls_missing_date_posted(csv_data)

    # Converting Series to DataFrame so we can see a tqdm progress bar.
    missing_posted_date_urls_frame = missing_posted_date_urls.to_frame()

    posted_dates = missing_posted_date_urls_frame[job_url_str].progress_apply(lambda url: get_date_posted_from_url(url))
    csv_data[date_posted_str] = posted_dates
    csv_data.to_csv(output_path)

In [488]:
files = get_csv_files(data_file_directory)

for file in files:
    filename = os.path.basename(file)
    print("Processing {}".format(filename))

    data = pd.read_csv(file, index_col=0)

    if has_missing_dates(data):
        fill_in_missing_posting_dates_form_csv(data, filename, "../data_fixed_dates")
    else: 
        print("Skipped {}".format(filename))

Processing 2021-10-14-chronicles_of_higher_ed.csv
Skipped 2021-10-14-chronicles_of_higher_ed.csv
Processing 2021-10-24-chronicles_of_higher_ed.csv
Skipped 2021-10-24-chronicles_of_higher_ed.csv
Processing 2021-11-01-chronicles_of_higher_ed.csv
Skipped 2021-11-01-chronicles_of_higher_ed.csv
Processing 2021-11-06-chronicles_of_higher_ed.csv
Skipped 2021-11-06-chronicles_of_higher_ed.csv
Processing 2021-11-13-chronicles_of_higher_ed.csv
Skipped 2021-11-13-chronicles_of_higher_ed.csv
Processing 2021-11-22-chronicles_of_higher_ed.csv
Skipped 2021-11-22-chronicles_of_higher_ed.csv
Processing 2021-11-28-chronicles_of_higher_ed.csv
Skipped 2021-11-28-chronicles_of_higher_ed.csv
Processing 2021-12-05-chronicles_of_higher_ed.csv
Skipped 2021-12-05-chronicles_of_higher_ed.csv
Processing 2021-12-14-chronicles_of_higher_ed.csv
Skipped 2021-12-14-chronicles_of_higher_ed.csv
Processing 2021-12-19-chronicles_of_higher_ed.csv
Skipped 2021-12-19-chronicles_of_higher_ed.csv
Processing 2021-12-27-chronicl